In [ ]:
# Setup basic variables
properties = LoadProperties()

# Declare username and password and dsn (data connection string)
un = "vector"
pw = "vector"
cs = "localhost/FREEPDB1"

# Connect to the database
try:
    conn23c = oracledb.connect(user=un, password=pw, dsn=cs)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")


In [ ]:
# In this section we will prepare using ConversationalRetrievalChain
# with memory. We use a LLM model and also pass conversation history
# in query. We will use Oracle vectorstore.

# Step 1 - here we create a function that creates a chain

def create_chain():
    embed_model = OCIGenAIEmbeddings(
        model_id=properties.getEmbeddingModelName(),
        service_endpoint=properties.getEndpoint(),
        compartment_id=properties.getCompartment(),
        auth_type="INSTANCE_PRINCIPAL"
    )

    vs = OracleVS.from_existing_index(embed_model,
                client=conn23c, table_name="DEMO_TABLE",
                distance_strategy=DistanceStrategy.DOT_PRODUCT)

    retrv = vs.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    llm = ChatOracle(
        model_id=properties.getModelName(),
        service_endpoint=properties.getEndpoint(),
        compartment_id=properties.getCompartment(),
        auth_type="INSTANCE_PRINCIPAL",
        model_kwargs={"max_tokens": 2000}
    )

    memory = ConversationBufferMemory(llm=llm,
                memory_key="chat_history", return_messages=True)

    qa = ConversationalRetrievalChain.from_llm(llm,
                retriever=retrv, memory=memory)

    return qa


In [ ]:
# Step 2 - here we declare a chat function

def chat(llm_chain, user_input):
    # generate a prediction for a prompt
    bot_response = llm_chain.invoke(user_input)
    return bot_response['bot_response']


In [ ]:
# Step 3 - here we setup streamlit text input and pass input text to chat function
# chat function returns the response and we print it.

if __name__ == "__main__":
    import streamlit as st

    st.subheader("Chatbot that answers your study questions")
    col1, col2 = st.columns([4,1])

    def initialize_session_state():
        if "llm_chain" not in st.session_state:
            st.session_state.llm_chain = create_chain()
            st.session_state.messages = []
        return st.session_state.llm_chain

    user_input = st.chat_input("Ask me a question about science")
    with col1:
        if user_input:
            llm_chain = initialize_session_state()
            bot_response = chat(llm_chain, user_input)
            print("BOT_RESPONSE>>>> ", bot_response)

            st.session_state.messages.append({"role": "chatbot", "content": bot_response})
            st.write("User: ", user_input)
            st.write("Chatbot: ", bot_response)

    for message in st.session_state.messages:
        st.chat_message("assistant").write(message["content"])

        st.chat_message("user").write(message["content"])


In [ ]:
!streamlit run chatbot-streamlit.py